In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
X = 40
my_game = rungame('TestGame', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()
Game.War

KeyboardInterrupt: 

In [6]:
len(Game.errors)

0

In [12]:
print(Game.War['Notes'].values[0])

Battle of Caercas

Belligerents:
Guilder el-Hadid (Point of Call Exchange) [A] vs. Marlae Roesone [D]

Strength:
[Guilder el-Hadid (Point of Call Exchange)]
 Scouts  1

[Marlae Roesone]
      Knights  1
      Archers  3
      Pikemen  3
 Artillerists  1
     Infantry  1

Casualties:
[Guilder el-Hadid (Point of Call Exchange)]
 Scouts  1
[Marlae Roesone]
[None]

Result: Marlae Roesone victory after 1 hours of fighting.


In [ ]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[0]

In [ ]:
if state[43] == 0 or state[44]==0 or state[94]==1 or state[98]==0:
    print('ok')

In [ ]:
Target = None
troops = []
provences = []

temp = self.Relationships[self.Relationships['Regent']==Regent]
temp = temp[temp['Other']==enemy]
try:
    animosity = -1*temp['Diplomacy'].values[0]
except:
    animosity = 0
if animosity <= 0:
    animosity = 1
animosity = 2*animosity - state[89] + 3*state[87]
# which should I attack?
found = 0
if Target != None:
    Target_Provence = Target
else:  # find target
    Target_Provence = ''
    s_dist = 9000
    temp = self.Provences[self.Provences['Regent']==enemy].copy()
    temp = pd.concat([temp[temp['Contested']==False][['Provence']], self.Troops[self.Troops['Regent']==enemy][['Provence']]], sort=False)
    if temp.shape[0] == 0:
        self.errors.append((Regent, 'Move-step2', self.Season, temp))
    else:
        temp['roll'] = np.random.randint(1, 100, temp.shape[0])
        temp = temp.sort_values('roll')
        for i, row in self.Provences[self.Provences['Regent']==enemy].copy().iterrows():
            try:
                new_dist = self.get_travel_cost(Regent, self.Troops[self.Troops['Regent']==Regent].iloc[0]['Provence'], row['Provence'], 'test')
            except:
                new_dist = 9000
            if new_dist < s_dist*0.75:  # some randomness
                s_dist = new_dist
                found = 1
                Target_Provence = row['Provence']
if len(troops) == 0 or len(provences)==0:  # assign troops
    temp = self.Troops[self.Troops['Regent']==Regent].copy()
    troops = []
    provences = []
    for i, row in temp.iterrows():
        if len(troops) <= animosity:
            troops.append(row['Type'])
            provences.append(row['Provence'])

success, reward, message = self.bonus_action_move_troops(Regent, troops, provences, Target_Provence)
reward = animosity + 5*state[87] + 5*state[43]

In [ ]:
Target_Provence